# PROCESAMIENTO DE IMÁGENES CANAL 13 Y VIENTO A 200 hPA

**Códigos para trabajar en entorno Google Colab, esto se hizo con el fin de no sobresaturar el equipo por el peso de los archivos de cada canal.
Se conectó todo a una cuenta drive y todos los archivos e imagenes quedaron almacenado en el directorio asignado.**

In [ ]:
!pip install cartopy
!pip install netCDF4
!pip install xarray
!pip install GOES
!pip install custom_color_palette

In [ ]:
# =========================================================
# 1. INSTALACIÓN Y MONTAJE (Ejecutar si es necesario)
# =========================================================
# Al ser entorno Colab, se tiene que vincular el libro a una cuenta drive para poder procesar los datos en la nube y no saturar nuestro equipo
# from google.colab import drive
# drive.mount('/content/drive')

import os
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from netCDF4 import Dataset
import xarray as xr
import custom_color_palette as ccp
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter

# =========================================================
# 2. CONFIGURACIÓN DE RUTAS
# =========================================================
ruta_base = "/content/drive/MyDrive/GOES19_ATACAMA_20250624_20250630/"
era5_file = "/content/drive/MyDrive/ERA5.nc"
# =========================================================
# CARPETA DE SALIDA DE GRÁFICOS
# =========================================================
out_dir = "/content/drive/MyDrive/GRAFICOS_GOES19_200hPa"
os.makedirs(out_dir, exist_ok=True)

domain = [-100, -30, -60, 15] # [Oeste, Este, Sur, Norte]
# =========================================================
# seleccionamos las carpetas que fueron nombradas con la fecha yq ue contienen todos los canales descargados
# =========================================================
targets = [
    "20250624_12",
    "20250625_12",
    "20250626_12",
    "20250627_12",
    "20250628_12",
    "20250629_12",
    "20250630_12"
]


In [ ]:

# =========================================================
# 3. configuramos la PALETA DE COLORES
# =========================================================
lower_colors = ['maroon','red','darkorange','#ffff00','forestgreen','cyan','royalblue',(148/255,0/255,211/255)]
lower_palette = [lower_colors, ccp.range(180.0,240.0,1.0)]
upper_colors = plt.cm.Greys
upper_palette = [upper_colors, ccp.range(240.0,330.0,1.0), [ccp.range(180.0,330.0,1.0),240.0,330.0]]
cmap, cmticks, norm, bounds = ccp.creates_palette([lower_palette, upper_palette], extend='both')

# =========================================================
# 4. CARGA DE ERA5 Y AJUSTE DE LONGITUD
# =========================================================
ds_era = xr.open_dataset(era5_file)
if ds_era.longitude.max() > 180:
    ds_era.coords['longitude'] = (ds_era.coords['longitude'] + 180) % 360 - 180
    ds_era = ds_era.sortby('longitude')

# =========================================================
# 5. BUCLE DE PROCESAMIENTO
# =========================================================
for target in targets:
    try:
        print(f"--- Procesando: {target} ---")
        target_dir = os.path.join(ruta_base, target)

        # Buscamos archivos del C13
        goes_files = [f for f in os.listdir(target_dir) if "C13" in f and f.endswith('.nc')]
        if not goes_files:
            continue

        GOES_FILE = os.path.join(target_dir, goes_files[0])

        # aquí se lee GOES con librería para obtener Lat/Lon directamente
        import GOES
        ds_goes = GOES.open_dataset(GOES_FILE)
        CMI, LonCen, LatCen = ds_goes.image('CMI', lonlat='center', domain=domain)

        #dejamos los datos en kelvin
        data_celsius = CMI.data  # EN KELVIN

        # --- SELECCIÓN DINÁMICA DE ERA5 ---
        fecha_dt = f"{target[:4]}-{target[4:6]}-{target[6:8]}T{target[9:11]}:00:00"
        dim_tiempo = 'valid_time' if 'valid_time' in ds_era.coords else 'time'
        dim_nivel  = 'pressure_level' if 'pressure_level' in ds_era.coords else 'level'

        era_step = ds_era.sel({dim_tiempo: fecha_dt, dim_nivel: 200}, method='nearest')

        # =========================================================
        # 6. GRAFICADO PROFESIONAL
        # =========================================================
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
        ax.set_extent(domain, crs=ccrs.PlateCarree())

        # Fondo y Capas
        ax.add_feature(cfeature.OCEAN, facecolor='#1a2a3a')
        ax.add_feature(cfeature.LAND, facecolor='black')
        ax.add_feature(cfeature.COASTLINE, edgecolor='white', linewidth=0.8)
        ax.add_feature(cfeature.BORDERS, edgecolor='cyan', linestyle=':', linewidth=0.5)

        # GOES IR (pcolormesh para exactitud con LonCen/LatCen)
        img = ax.pcolormesh(LonCen.data, LatCen.data, data_celsius,
                            cmap=cmap, norm=norm, shading='auto', transform=ccrs.PlateCarree())

        # Viento ERA5 (Amarillo para contraste)
        skip = 15
        ax.quiver(era_step.longitude.values[::skip], era_step.latitude.values[::skip],
                  era_step.u.values[::skip, ::skip], era_step.v.values[::skip, ::skip],
                  color="yellow", scale=700, transform=ccrs.PlateCarree(),
                  zorder=5, width=0.003)

        # formato de Ejes
        gl = ax.gridlines(draw_labels=True, linewidth=0.3, color='white', alpha=0.3)
        gl.top_labels = gl.right_labels = False
        gl.xformatter = LongitudeFormatter(); gl.yformatter = LatitudeFormatter()

        # Barra de Colores y Títulos
        cb = fig.colorbar(img, orientation='horizontal', pad=0.05, fraction=0.04)
        cb.set_label("Temperatura de Brillo (°K)", weight='bold')
        plt.title(f"GOES-19 Ch13 + Viento 200 hPa\nFecha: {target} UTC", fontweight='bold', loc='left')

        # =========================================================
        # GUARDAMOS LA FIGURA EN EL DRIVE
        # =========================================================
        out_file = os.path.join(out_dir, f"GOES19_C13_200hPa_{target}.png")

        plt.savefig(out_file, dpi=300, bbox_inches="tight")
        plt.close(fig)

        print(f"✔ Guardado: {out_file}")



    except Exception as e:
        print(f"❌ Error en {target}: {e}")




--- Procesando: 20250624_12 ---
✔ Guardado: /content/drive/MyDrive/GRAFICOS_GOES19_200hPa/GOES19_C13_200hPa_20250624_12.png
--- Procesando: 20250625_12 ---
✔ Guardado: /content/drive/MyDrive/GRAFICOS_GOES19_200hPa/GOES19_C13_200hPa_20250625_12.png
--- Procesando: 20250626_12 ---
✔ Guardado: /content/drive/MyDrive/GRAFICOS_GOES19_200hPa/GOES19_C13_200hPa_20250626_12.png
--- Procesando: 20250627_12 ---
✔ Guardado: /content/drive/MyDrive/GRAFICOS_GOES19_200hPa/GOES19_C13_200hPa_20250627_12.png
--- Procesando: 20250628_12 ---
✔ Guardado: /content/drive/MyDrive/GRAFICOS_GOES19_200hPa/GOES19_C13_200hPa_20250628_12.png
--- Procesando: 20250629_12 ---
✔ Guardado: /content/drive/MyDrive/GRAFICOS_GOES19_200hPa/GOES19_C13_200hPa_20250629_12.png
--- Procesando: 20250630_12 ---
✔ Guardado: /content/drive/MyDrive/GRAFICOS_GOES19_200hPa/GOES19_C13_200hPa_20250630_12.png
